![](https://www.dii.uchile.cl/wp-content/uploads/2021/06/Magi%CC%81ster-en-Ciencia-de-Datos.png)

# **Proyecto 1 - MDS7202 Laboratorio de Programación Científica para Ciencia de Datos 📚**

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

*Por favor, lean detalladamente las instrucciones de la tarea antes de empezar a escribir.*

### Equipo:

- Michelle Avendaño
- Claudia Navarro


### Link de repositorio de GitHub: `\<http://....\>`

Fecha límite de entrega 📆: 06 de Noviembre de 2023.

----

## Reglas

- **Grupos de 2 personas.**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Estrictamente prohibida la copia.
- Pueden usar cualquier material del curso que estimen conveniente.

<div style="text-align: center;">
    <img src="https://worldskateamerica.org/wp-content/uploads/2023/07/SANTIAGO-2023-1-768x153.jpg" alt="Descripción de la imagen">
</div>

En un Chile azotado por un profundo caos político-económico y el resurgimiento de programas de televisión de dudosa calidad, todas las miradas y esperanzas son depositadas en el éxito de un único evento: Santiago 2023. La nación necesitaba desesperadamente un respiro, y los Juegos de Santiago 2023 prometían ser una luz al final del túnel.

El Presidente de la República -conocido en las calles como Bombín-, consciente de la importancia de este evento para la revitalización del país, decide convocar a usted y su equipo en calidad de expertos en análisis de datos y estadísticas. Con gran solemnidad, el presidente les encomienda una importante y peligrosa: liderar un proyecto que permitiera caracterizar de forma automática y eficiente los datos generados por estos magnos juegos. Para esto, el presidente le destaca que la solución debe considerar los siguientes puntos:
- Caracterización automática de los datos
- La solución debe ser compatible con cualquier dataset
- Se les facilita el dataset *olimpiadas.parquet*, el cual recopila data de diferentes juegos olímpicos realizados en los últimos años

In [1]:
#importamos librería
import pandas as pd
import numpy as np
import os
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import re
import shutil

In [2]:
data = pd.read_parquet('olimpiadas.parquet')
data

,ID,Name,Sex,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,age-height-weight
0,1,A Dijiang,M,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None,24.0*180.0?80.0
1,2,A Lamusi,M,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None,23.0(170.0?60.0
2,3,Gunnar Nielsen Aaby,M,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None,24.0(nan?nan
3,4,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,34.0:nan?nan
4,5,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None,21.0(185.0?82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,None,29.0:179.0?89.0
271112,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",None,27.0:176.0?59.0
271113,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",None,27.0*176.0?59.0
271114,135571,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,None,30.0(185.0?96.0


## 1.1 Creación de `Profiler` Class (4.0 puntos)

Cree la clase `Profiler`. Como mínimo, esta debe tener las siguientes funcionalidades:

1. El método constructor, el cual debe recibir los datos a procesar en formato `Pandas DataFrame`. Además, este método debe generar una carpeta en su directorio de trabajo con el nombre `EDA_fecha`, donde `fecha` corresponda a la fecha de ejecución en formato `DD-MM-YYYY`.

2. El método `summarize`, el cual debe caracterizar las variables del Dataset. Como mínimo, se espera que su método pueda:
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Reportar el tipo de variable
    - Reportar el número y/o porcentaje de valores únicos de la variable
    - Reportar el número y/o porcentaje de valores nulos
    - Si la variables es numérica:
        - Reportar el número y/o porcentaje de valores cero, negativos y outliers
        - Reportar estadística descriptiva como el valor mínimo, máximo, promedio y los percentiles 25, 50, 75 y 100
   - Levantar una alerta en caso de encontrar alguna anomalía fuera de lo común (el criterio debe ser ajustable por el usuario)
   - Guardar sus resultados en el directorio `EDA_fecha/summary.txt`. El archivo debe separar de forma clara y ordenada los resultados de cada punto.

3. El método `plot_vars`, el cual debe graficar la distribución e interraciones de las variables del Dataset. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/plots`
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Para las variables numéricas:
        - Genere un gráfico de distribución de densidad
        - Grafique la correlación entre las variables
    - Para las variables categóricas:
        - Genere un histograma de las top N categorías (N debe ser un parámetro ajustable)
        - Grafique el coeficiente V de Cramer entre las variables
    - Guardar cada gráfico generado en la carpeta `EDA_fecha/plots` en formato `.pdf` y bajo el naming `variable.pdf`, donde `variable` es el nombre de la variable de interés
    
4. El método `clean_data`, el cual debe limpiar los datos para que luego puedan ser procesados. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/clean_data`
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Drop de valores duplicados
    - Implementar como mínimo 2 técnicas para tratar los valores nulos, como:
        - Drop de valores nulos
        - Imputar valores nulos con alguna técnica de imputación
        - Funcionalidad para escoger entre una técnica y la otra.
    - Una de las columnas del dataframe presenta datos *no atómicos*. Separe dicha columna en las columnas que la compongan.
        - Hint: ¿Qué caracteres permiten separar una columna de otra?
        - Para las pruebas con el dataset nuevo, puede esperar que exista al menos una columna con este tipo de problema. Asuma que los separadores serán los mismos, aunque el número de columnas a separar puede ser distinto.
    - Deberían usar `FunctionTransformer`.
    - Guardar los datos procesados en formato `.csv` en el path `EDA_fecha/clean_data/data.csv`

5. El método `scale`, el cual debe preparar adecuadamente los datos para luego ser consumidos por algún tipo de algoritmo. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/scale`
    - Procesar de forma adecuada los datos numéricos y categóricos:
        - Su método debe recibir las técnicas de escalamiento como argumento de entrada (utilizar solo técnicas compatibles con el framework de `sklearn`)
        - Para los atributos numéricos, se transforme los datos con un escalador logarítmico y un `MinMaxScaler`
        - Asuma que no existen datos ordinales en su dataset
    - Guardar todo este procesamiento en un `ColumnTransformer`.
    - Guardar los datos limpios y transformados en formato `.csv` en el path `EDA_fecha/process/scaled_features.csv`

6. El método `make_clusters`, el cual debe generar clusters de los datos usando algún algoritmo de clusterización. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/clusters`
    - Generar un estudio del codo donde señale la cantidad de clusters optimos para el desarrollo.
    - Su método debe recibir el algoritmo de clustering como argumento de entrada (utilizar solo algoritmos compatibles con el framework de `sklearn`).
    - No olvide pre procesar adecuadamente los datos antes de implementar la técnica de clustering.
    - En este punto es espera que generen un `Pipeline` de sklearn. Además, su método debería usar lo construido en los puntos 4 y 5.
    - Su método debe ser capaz de funcionar a partir de datos crudos (se descontará puntaje de lo contrario).
    - Una vez generado los clusters, proyecte los datos a 2 dimensiones usando su técnica de reducción de dimensionalidad favorita y grafique los resultados coloreando por cluster.
    - Guardar los datos con su respectivo cluster en formato `.csv` en el path `EDA_fecha/clusters/data_clusters.csv`. Guarde también los gráficos generados en el mismo path.

7. El método `detect_anomalies`, el cual debe detectar anomalías en los datos. Como mínimo, se espera que su método pueda:

    - Crear la carpeta `EDA_fecha/anomalies`
    - Implementar alguna técnica de detección de anomalías.
    - Al igual que el punto anterior, su método debe considerar los siguientes puntos:
        - No olvide pre procesar de forma adecuada los datos antes de implementar la técnica de detección de anomalía.
        - En este punto es espera que generen un `Pipeline` de sklearn. Además, su método debería usar lo construido en los puntos 4 y 5.
        - Su método debe ser capaz de funcionar a partir de datos crudos (se descontará puntaje de lo contrario).
        - Su método debe recibir el algoritmo como argumento de entrada
        - Una vez generado las etiquetas, proyecte los datos a 2 dimensiones y grafique los resultados coloreando por las etiquetas predichas por el detector de anomalías
    - Guardar los datos con su respectiva etiqueta en formato `.csv` en el path `EDA_fecha/anomalies/data_anomalies.csv`. Guarde también los gráficos generados en el mismo path.

8. El método `profile`, el cual debe ejecutar todos los métodos anteriores.

9. Crear el método `clearGarbage` para eliminar las carpetas/archivos creados/as por la clase `Profiler`.

Algunas consideraciones generales:
- Su clase será testeada con datos tabulares diferentes a los provistos. No desarrollen código *hardcodeado*: su clase debe ser capaz de funcionar para **cualquier** dataset.
- Aplique todo su conocimiento sobre buenas prácticas de programación: se evaluará que su código sea limpio y ordenado.
- Recuerden documentar cada una de las funcionalidades que implementen.
- Recuerden adjuntar sus `requirements.txt` junto a su entrega de proyecto. **El código que no se pueda ejecutar por imcompatibilidades de librerías no será corregido.**

In [41]:

class Profiler():

    def __init__(self, data: pd.DataFrame):
        """
        Inicializa la clase Profiler con el DataFrame proporcionado y crea una carpeta
        para almacenar los resultados del EDA con la fecha actual como referencia.

        Args:
            data (pd.DataFrame): El DataFrame a analizar.
        """
        self.data = data

        # Crear carpeta con el formato EDA_fecha
        self.folder_name = "EDA_" + datetime.datetime.now().strftime('%d-%m-%Y')
        if not os.path.exists(self.folder_name):
            os.makedirs(self.folder_name)

###SUMMARIZE###
    def summarize(self, cols=None, factor_outlier=1.5):
        """
        Resumen de estadísticas descriptivas para las columnas del DataFrame.
        Puede calcular y alertar sobre la presencia de valores atípicos usando el método del rango intercuartílico (IQR).

        Args:
            cols (list, optional): Columnas específicas a resumir. Si es None, resume todas las columnas.
            factor_outlier (float): Factor para identificar valores atípicos basado en el IQR.
        """
        if cols:
            df = self.data[cols]
        else:
            df = self.data

        summary = {}
        for col in df.columns:
            summary[col] = {}
            summary[col]['tipo'] = df[col].dtype
            summary[col]['valores_unicos'] = df[col].nunique()
            summary[col]['valores_nulos'] = df[col].isna().sum()


            if df[col].dtype in ['int64', 'float64']:
                summary[col]['valores_cero'] = (df[col] == 0).sum()
                summary[col]['valores_negativos'] = (df[col] < 0).sum()

                # Outliers
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                outliers = df[(df[col] < (Q1 - factor_outlier * IQR)) | (df[col] > (Q3 + factor_outlier * IQR))] #aqui influye el valor del factor
                if not outliers.empty:
                    print(f"¡Alerta! Se encontraron {len(outliers)} outliers en la columna '{col}'")
                summary[col]['outliers'] = len(outliers)

                # Estadísticas descriptivas
                summary[col]['min'] = df[col].min()
                summary[col]['25%'] = df[col].quantile(0.25)
                summary[col]['50%'] = df[col].quantile(0.50)
                summary[col]['75%'] = df[col].quantile(0.75)
                summary[col]['max'] = df[col].max()
        
        # Verificar duplicados
        duplicates_count = len(self.data) - len(self.data.drop_duplicates())
        if duplicates_count > 0:
            print(f"¡ALERTA! Se encontraron {duplicates_count} valores duplicados en el conjunto de datos.")

        #Resumen
        with open(f'{self.folder_name}/summary.txt', 'w') as f:
            for k, v in summary.items():
                f.write(f"{k}:\n")
                for key, val in v.items():
                    f.write(f"  {key}: {val}\n")
                f.write("\n")
###PLOT_VARS###


    def plot_vars(self, cols=None, top_n=10):
        """
        Método que genera y guarda visualizaciones para las distribuciones de las variables
        y sus correlaciones, tanto para variables numéricas como categóricas.

        Para las variables numéricas, crea gráficos de densidad de distribución y un mapa de calor
        de correlación. Para las variables categóricas, genera histogramas de las principales
        categorías y calcula el coeficiente V de Cramer entre pares de variables categóricas.

        Args:
            cols (list, optional): Lista de columnas para las que se generarán las visualizaciones.
                Si se omite, se utilizarán todas las columnas del DataFrame.
            top_n (int, optional): Número de principales categorías a visualizar en los histogramas
                para variables categóricas. Por defecto es 10.

        Notas:
            - Los gráficos generados se guardarán en el subdirectorio 'plots' dentro de la carpeta
              correspondiente al EDA con la fecha actual.
            - Los archivos de gráficos se guardan en formato PDF.
        """
        # Crear carpeta si no existe
        plots_folder = f'{self.folder_name}/plots'
        os.makedirs(plots_folder, exist_ok=True)

        if cols:
            df = self.data[cols]
        else:
            df = self.data

        # Graficar distribución y correlación para variables numéricas
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_cols:
            plt.figure(figsize=(10,6))
            sns.kdeplot(df[col], fill=True)
            plt.title(f'Densidad de distribución de {col}')
            plt.savefig(f'{plots_folder}/{col}.pdf')
            plt.close()

        if len(numeric_cols) > 1:
            plt.figure(figsize=(10,6))
            sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
            plt.title('Correlación de las variables numéricas')
            plt.savefig(f'{plots_folder}/correlation.pdf')
            plt.close()

      ###Categóricas###

        #Top N
        cat_cols = df.select_dtypes(include=['object']).columns
        for col in cat_cols:
            plt.figure(figsize=(10,6))
            df[col].value_counts().head(top_n).plot(kind='bar')
            plt.title(f'Top {top_n} categorías de {col}')
            plt.savefig(f'{plots_folder}/{col}_histogram.pdf')
            plt.close()


        # Crear una matriz de correlación para las variables categóricas
        corr_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)
        for col1 in cat_cols:
            for col2 in cat_cols:
                if col1 != col2:
                   # Verificación para saltar combinaciones con alta cardinalidad
                    if df[col1].nunique() * df[col2].nunique() > 300000:
                        corr_matrix.loc[col1, col2] = np.nan
                        continue
                    
                    confusion_matrix = pd.crosstab(df[col1], df[col2])
                    chi2 = chi2_contingency(confusion_matrix)[0]
                    n = confusion_matrix.sum().sum()
                    phi2 = chi2 / n
                    r, k = confusion_matrix.shape
                    # Agregar comprobación para dimensiones negativas o cero
                    if min(r-1, k-1) == 0:
                        corr_matrix.loc[col1, col2] = 0
                    else:
                        phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
                        rcorr = r - ((r - 1) ** 2) / (n - 1)
                        kcorr = k - ((k - 1) ** 2) / (n - 1)
                        cramers_v = np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))
                        corr_matrix.loc[col1, col2] = cramers_v
                        
        # Graficar la matriz de correlación
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix.astype(float), annot=True, cmap='coolwarm', center=0)
        plt.title('Coeficiente de Cramer\'s V entre variables categóricas')
        plt.savefig(f'{plots_folder}/cramer_v_matrix.pdf')
        plt.close()


###CLEAN DATA###

    def _split_non_atomic(self, data, separators=[':', "(", "?", '*']):
            """
            Función para dividir las columnas que contienen múltiples valores en una sola celda.

            Args:
                data (pd.DataFrame): El DataFrame con posibles columnas no atómicas.
                separators (list): Lista de separadores utilizados para dividir las columnas.

            Returns:
                pd.DataFrame: DataFrame con las columnas divididas y valores convertidos a numéricos donde sea posible.
            """

            # Escapar los separadores para regex
            separators_regex = '|'.join(map(re.escape, separators))

            # Identificar columnas no numéricas con separadores
            non_atomic_columns = [
                column for column in data.columns if not pd.api.types.is_numeric_dtype(data[column]) and
                data[column].str.contains(separators_regex).any() and not data[column].str.contains("\)").any()
            ]

            # Procesar cada columna no atómica
            for column in non_atomic_columns:
                # Dividir la columna en múltiples columnas usando los separadores
                split_data = data[column].str.split(separators_regex, expand=True)

                # Asignar nuevos nombres a las columnas divididas
                split_columns = [f'{column}_{i}' for i in range(split_data.shape[1])]
                split_data.columns = split_columns

                # Convertir a numérico si es posible
                for split_column in split_columns:
                    split_data[split_column] = pd.to_numeric(split_data[split_column], errors='coerce')

                # Eliminar la columna original y unir las nuevas columnas al DataFrame
                data = data.drop(column, axis=1).join(split_data)

            # Reemplazar cadenas 'nan' con np.nan para uniformidad
            data.replace('nan', np.nan, inplace=True)
            return data


    def _cleaning_operations(self, data, method, nan_threshold=0.5, cols_to_clean=None):
        """
          Función para realizar operaciones de limpieza en el DataFrame, como eliminar columnas con
          un alto porcentaje de valores NaN o imputar valores faltantes.

          Args:
              data (pd.DataFrame): DataFrame a limpiar.
              method (str): Método de limpieza ('drop' o 'impute').
              nan_threshold (float): Umbral para la proporción de NaNs permitidos antes de eliminar una columna.
              cols_to_clean (list): Columnas específicas a limpiar. Si es None, limpia todo el DataFrame.

          Returns:
              pd.DataFrame: DataFrame limpio.
        """
        #Si es que hay columnas en específico que limpiar
        if cols_to_clean:
            df_to_clean = data[cols_to_clean].drop_duplicates()
        else:
            df_to_clean = data.drop_duplicates()
        #Eliminar columnas con muchos NAN
        threshold = len(df_to_clean) * nan_threshold
        df_to_clean = df_to_clean.dropna(thresh=threshold, axis=1)

        #Métodos para los NAN, de defecto es drop
        if method == "drop":
            df_to_clean.dropna(inplace=True)
        elif method == "impute":
            for col in df_to_clean.columns:
                if pd.api.types.is_numeric_dtype(df_to_clean[col]):
                    df_to_clean[col].fillna(df_to_clean[col].median(), inplace=True)
                else:
                    df_to_clean[col].fillna(df_to_clean[col].mode()[0], inplace=True) #Para las categóricas
        
        # Eliminar duplicados finales
        df_to_clean = df_to_clean.drop_duplicates()

        if cols_to_clean:
            # Reemplazar las columnas originales con las limpiadas en el DataFrame completo
            data.update(df_to_clean)
            return data
        else:
            return df_to_clean

    def clean_data(self, cols_to_clean=None, method="drop", nan_threshold=0.5):

        """
        Método  para limpiar el DataFrame. Divide primero las columnas no atómicas y luego realiza
        las operaciones de limpieza especificadas.

        Args:
            cols_to_clean (list): Columnas específicas a limpiar. Si es None, limpia todo el DataFrame.
            method (str): Método de limpieza a utilizar ('drop' o 'impute').
            nan_threshold (float): Umbral para la proporción de NaNs permitidos antes de eliminar una columna.

        Returns:
            pd.DataFrame: DataFrame limpio.
        """
        clean_data_folder = f'{self.folder_name}/clean_data'
        os.makedirs(clean_data_folder, exist_ok=True)

        # Aplicar _split_non_atomic a todo el DataFrame
        split_transformer = FunctionTransformer(self._split_non_atomic)
        df_split = split_transformer.transform(self.data.copy())

        # Aplicar _cleaning_operations solo a las columnas especificadas
        clean_transformer = FunctionTransformer(
            self._cleaning_operations,
            kw_args={'method': method, 'nan_threshold': nan_threshold, 'cols_to_clean': cols_to_clean}
        )
        df_clean = clean_transformer.transform(df_split)

        df_clean.to_csv(f'{clean_data_folder}/data.csv', index=False)
        return df_clean

###SCALE###



###MAKE CLUSTERS###



##DETECT ANOMALIES###






###PROFILE###
    def profile(self):
            """
            Ejecuta todos los métodos de la clase.
            """
            print("Iniciando el perfilado de los datos.")

            
            # 1. Caracterizar las variables
            print("Caracterizando variables...")
            self.summarize()

            # 2. Graficar la distribución e interacciones de las variables
            print("Graficando variables...")
            self.plot_vars()

            # 3. Limpiar los datos
            print("Limpiando datos...")
            self.clean_data()

            # 4. Preparar los datos para algoritmos de aprendizaje automático
            #print("Escalando características...")
            #self.scale()

            # 5. Crear clusters
            #print("Generando clusters...")
            #self.make_clusters()

            # 6. Detectar anomalías
            #print("Detectando anomalías...")
            #self.detect_anomalies()

            print("Listo ✨")




###CLEAR GARBAGE###
    def clearGarbage(self):
            """
            Elimina la carpeta creada con todas las subcarpetas
            """

            try:
                shutil.rmtree(self.folder_name)
                print(f"La carpeta {self.folder_name} ha sido eliminada.")
            except OSError as e:
                print(f"Error: {e.strerror}")





## 1.2 Caracterizar datos de Olimpiadas (2.0 puntos)

A partir de la clase que hemos desarrollado previamente, procederemos a realizar un análisis exhaustivo de los datos proporcionados en el enunciado. Este análisis se presentará en forma de un informe contenido en el mismo Jupyter Notebook y abordará los siguientes puntos:

1. Introducción
    - Se proporcionará una breve descripción del problema que estamos abordando y se explicará la metodología que se seguirá.

Elaborar una breve introducción con todo lo necesario para entender qué realizarán durante su proyecto. La idea es que describan de manera formal el proyecto con sus propias palabras y logren describir algunos aspectos básicos tanto del dataset como del análisis a realizar sobre los datos.

Por lo anterior, en esta sección ustedes deberán ser capaces de:

- Describir la tarea asociada al dataset.
- Describir brevemente los datos de entrada que les provee el problema.
- Plantear hipótesis de cómo podrían abordar el problema.

2. Análisis del EDA (Análisis Exploratorio de Datos)
    - Se discutirán las observaciones y conclusiones obtenidas acerca de los datos proporcionados. A lo largo de su respuesta, debe responder preguntas como:
        - ¿Como se comportan las variables numéricas? ¿y las categóricas?
        - ¿Existen valores nulos en el dataset? ¿En qué columnas? ¿Cuantos?
        - ¿Cuáles son las categorías y frecuencias de las variables categóricas?
        - ¿Existen datos duplicados en el conjunto?
        - ¿Existen relaciones o patrones visuales entre las variables?
        - ¿Existen anomalías notables o preocupantes en los datos?
3. Creación de Clusters y Anomalías
    - Se justificará la elección de los algoritmos a utilizar y sus hiperparámetros. En el caso de clustering, justifique además el número de clusters.
    
4. Análisis de Resultados
    - Se examinarán los resultados obtenidos a partir de los clústers y anomalías generadas. ¿Se logra una separación efectiva de los datos? Entregue una interpretación de lo que representa cada clúster y anomalía.
5. Conclusión
    - Se resumirán las principales conclusiones del análisis y se destacarán las implicaciones prácticas de los resultados obtenidos.

**INTRODUCCIÓN**

Chile se encuentra en un momento crítico de su historia, caracterizado por la agitación política y desafíos económicos. En este contexto, los Juegos de Santiago 2023 representan una oportunidad crucial para la nación. Estos juegos no solo prometen destacar a nivel internacional, sino también ofrecer un respiro bienvenido para la sociedad chilena.


El presidente de la República, conocido como "Bombín" por la población, ha convocado a un equipo de expertos en análisis de datos y estadísticas para liderar un proyecto esencial. La hipótesis fundamental es que una caracterización detallada de los datos generados por los juegos permitirá tomar decisiones informadas, identificar patrones clave y abordar desafíos anticipadamente.

Sin embargo, un desafío evidente en esta tarea es la presencia de valores nulos (NaN) en los datos, lo que dificulta su análisis y la obtención de información útil. Además, la estructura de la tabla, no siempre atómica, con datos en un formato que requiere procesamiento, plantea un reto adicional para el análisis.


Los datos en cuestión, como se muestra en la siguiente tabla, incluyen información variada, desde detalles sobre los atletas y sus logros hasta datos demográficos como edad, altura y peso.

In [4]:
data

,ID,Name,Sex,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,age-height-weight
0,1,A Dijiang,M,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None,24.0*180.0?80.0
1,2,A Lamusi,M,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None,23.0(170.0?60.0
2,3,Gunnar Nielsen Aaby,M,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None,24.0(nan?nan
3,4,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,34.0:nan?nan
4,5,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None,21.0(185.0?82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,None,29.0:179.0?89.0
271112,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",None,27.0:176.0?59.0
271113,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",None,27.0*176.0?59.0
271114,135571,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,None,30.0(185.0?96.0


A lo largo de este proyecto, se creará una clase Profiler versátil para analizar los datos, incluyendo la caracterización de variables, generación de gráficos, limpieza de datos y detección de anomalías. Se realizará un análisis exhaustivo de los datos de las Olimpiadas proporcionados en el archivo "olimpiadas.parquet", abordando aspectos como valores nulos, duplicados y patrones visuales. Además, se explorarán clusters y anomalías, evaluando su relevancia en el contexto de los Juegos de Santiago 2023.


El objetivo es proporcionar una base sólida para la toma de decisiones y contribuir al éxito de un evento crucial para la revitalización de Chile.

**Análisis Exploratorio de Datos**

Para esta sección, se analizará la data original (sin hacer una muestra),es decir, antes de realizar la limpieza de datos. 

In [42]:
#Se crea una instancia y se realiza el método profile  con las entradas default
instancia = Profiler(data)
instancia.summarize()
instancia.clean_data()

¡Alerta! Se encontraron 380 outliers en la columna 'Year'
¡ALERTA! Se encontraron 668 valores duplicados en el conjunto de datos.


,ID,Name,Sex,Team,NOC,Games,Year,Season,City,Sport,Event,age-height-weight_0,age-height-weight_1,age-height-weight_2
0,1,A Dijiang,M,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,24.0,180.0,80.0
1,2,A Lamusi,M,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,23.0,170.0,60.0
4,5,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,21.0,185.0,82.0
5,5,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",21.0,185.0,82.0
6,5,Christine Jacoba Aaftink,F,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,25.0,185.0,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,29.0,179.0,89.0
271112,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",27.0,176.0,59.0
271113,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",27.0,176.0,59.0
271114,135571,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,30.0,185.0,96.0


Tras analizar el resumen de los datos, se destacan varios aspectos clave en el conjunto de datos. Este conjunto tiene 668 valores duplicados y consta de 13 variables, siendo únicamente dos de ellas de naturaleza numérica. La variable "YEAR" presenta 380 valores atípicos según el método IQR, lo que sugiere posibles discrepancias en los años registrados. Sin embargo, dado que "YEAR" representa el año en que se llevaron a cabo los eventos deportivos, es comprensible que haya outliers debido al amplio rango temporal cubierto por los datos.

Por otro lado, la variable "Medal" es la única que contiene valores nulos, totalizando 231,333 valores faltantes. Dado que la cantidad de filas en la base de datos es de 271,116, esta cantidad de valores nulos es considerable y requerirá atención.

Finalmente, se observa que la variable denominada "age-height-weight," a pesar de su nombre, se clasifica como categórica, lo que resulta inusual. Es necesario investigar más a fondo los datos en esta columna para entender su contenido y su relación con la edad, altura y peso.

En lo que respecta a las distribuciones de las variables numéricas, se destacan varios hallazgos significativos. La variable "YEAR" abarca un amplio rango de años, y se observa que alrededor de la década de 1940, hubo una disminución en la participación en los juegos, mientras que a partir de la década de 2000, la participación ha experimentado un notable aumento.

En cuanto al identificador "ID," se advierte que la mayoría de los jugadores han participado en múltiples ocasiones, indicando una reiterada presencia de atletas en los juegos. Es relevante señalar que en el gráfico de correlación se muestra una baja correlación entre estas variables, lo cual es coherente, ya que "ID" funciona exclusivamente como un identificador único.

Prosiguiendo con los histogramas de las variables categóricas, se destaca que el equipo con mayor presencia en los juegos es Estados Unidos, y, de manera consistente, este país es el que más se repite en la variable "NOC." Además, el atletismo sobresale como el deporte con mayor participación, y el evento con la mayor cantidad de participantes es el fútbol. Respecto a las sedes de los juegos, Londres ha sido la ciudad que ha albergado a la mayor cantidad de deportistas. Además, se observa que la mayoría de los eventos se han realizado en la temporada de verano (no existen categorías para otoño o primavera) y el verano del año 2000 destaca como uno de los más concurridos.

En cuanto al género de los deportistas, prevalece el sexo masculino. Entre los jugadores, Robert Tait McKenzie destaca como el más frecuente en la base de datos. 

En cuanto a la variable "MEDAL", se observa que la distribución de medallas (oro, plata, bronce) es similar, sin embargo, hay más medallas de bronce que de plata lo que no es lógico. Este patrón podría explicarse por la alta cantidad de valores nulos en la variable "MEDAL". Además, como se mencionó previamente, la variable "age-height-weight" carece de sentido como una variable categórica y requerirá una división adecuada durante el proceso de limpieza de datos. 

En relación al gráfico del "Coeficiente de Cramer's V entre variables categóricas," se observa que existe una correlación perfecta (igual a 1) entre "TEAM" y "NOC," así como entre "SEASON" y "GAMES," "CITY," "SPORT," y "EVENT." También se observa una alta correlación entre "CITY" y "GAMES," entre "SPORT" y "EVENT," y entre "EVENT" y "SEX." Esta correlación alta tiene sentido, dado que algunos deportes están estrechamente vinculados al sexo masculino o femenino.

Por otro lado, "MEDAL" muestra una correlación muy baja con las demás variables, pero es fundamental destacar que esta baja correlación podría atribuirse a la gran cantidad de valores nulos en la variable. Es importante señalar que debido a las limitaciones del computador, para la variable "NAME" no fue posible calcular todas las correlaciones debido a la gran cantidad de nombres únicos (lo mismo ocurre con "age-height-weight").

Ahora se procederá a analizar los cambios provocados por el método CLEAN_DATA.

In [43]:
data_limpia = pd.read_csv("EDA_06-11-2023\clean_data\data.csv")
instancia_2 =Profiler(data_limpia)
instancia_2.summarize()
instancia_2.plot_vars()

¡Alerta! Se encontraron 3020 outliers en la columna 'Year'
¡Alerta! Se encontraron 4626 outliers en la columna 'age-height-weight_0'
¡Alerta! Se encontraron 1310 outliers en la columna 'age-height-weight_1'
¡Alerta! Se encontraron 3174 outliers en la columna 'age-height-weight_2'


Con la limpieza de datos se eliminaron todas las filas que contenían valores nulos, así como los duplicados, dejando un total de 206152 filas. Cabe destacar que durante este proceso, se eliminó la columna "MEDAL" debido a la gran cantidad de datos nulos que contenía. Además, la variable "age-height-weight" se dividió en tres columnas separadas, que representan la edad, la altura y el peso respectivamente. Estas tres nuevas variables presentan 4626, 1310 y 3174 outliers respectivamente. Es importante resaltar que ni antes de la limpieza ni después de esta, se encontraron valores negativos en los datos.

Analizando las distribuciones de las nuevas variables, "age-height-weight_0", que corresponde a la edad, se nota una concentración predominante alrededor de los 25 años. La variable "age-height-weight_1", referente a la altura, revela tres picos significativos alrededor de los 1.70 m, 1.75 m y 1.85 m. En cuanto a "age-height-weight_2", asociada al peso, la mayoría de los registros se aglomeran alrededor de los 76 kg. Además, existe una correlación notable entre la altura y el peso, con un coeficiente de 0.8, lo que indica una fuerte relación proporcional entre estas dos medidas físicas, lo que tiene sentido.

Con respecto a las variables categóricas, se han percibido algunos cambios en los histogramas. En la variable 'EVENT', el hockey ha emergido como la disciplina líder, seguido de cerca por el fútbol. Por otra parte, en términos de participaciones, Heikki Ilmari Savolainen resalta como el deportista con mayor número de registros en el conjunto de datos actualizado. En cuanto a los coeficientes de cramers, los coeficientes se mantienen en un rango similar.